# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева, НИУ ВШЭ*

## Управление браузером с Selenium: продолжение

## Часть 1: пример скроллинга с Selenium 

Импортируем все необходимые модули и функции:

In [ ]:
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from getpass import getpass
from bs4 import BeautifulSoup
from time import sleep

Сразу запрашиваем и сохраняем свой пароль от ВКонтакте:

In [ ]:
my_password = getpass()

Функция для извлечения имен пользователей и ссылок на их аккаунты (с прошлого практикума):

In [ ]:
def get_person(d):
    name = d.text
    href = d.find("a")["href"]
    full_href = "https://vk.com" + href
    return name, full_href

Запускаем браузер через Selenium:

In [ ]:
br = wd.Chrome(executable_path='/Users/allat/Desktop/chromedriver')

Находим поле для логина и заполняем:

In [ ]:
br.get("https://vk.com/")
br.implicitly_wait(3)

login = br.find_element(By.ID, "index_email")
login.send_keys("allatambov@mail.ru")
br.implicitly_wait(3)
login.send_keys(Keys.ENTER)

Находим поле для пароля и заполняем (к слову, `Keys.ENTER` и `Keys.RETURN` – одно и то же, одна и та жа клавиша для перехода на новую строку):

In [ ]:
password = br.find_element(By.NAME, "password")
password.send_keys(my_password)
br.implicitly_wait(3)
password.send_keys(Keys.RETURN)

Переходим на страницу для поиска людей:

In [ ]:
br.get("https://vk.com/friends")

Выставляем все фильтры для поиска сразу (код с прошлого практикума, только в одной ячейке):

In [ ]:
search = br.find_element(By.LINK_TEXT, "Поиск друзей")
search.click() 
br.implicitly_wait(2)

pars = br.find_element(By.ID, "friends_filters_block")
pars.click()
br.implicitly_wait(2)

city = br.find_element(By.ID, "cCity")
city_inp = city.find_element(By.TAG_NAME, "input")
city_inp.send_keys("Москва") 
br.implicitly_wait(3)
city_inp.send_keys(Keys.RETURN)

sex = br.find_element(By.ID, "cSex")
values = sex.find_elements(By.TAG_NAME, "div")
values[1].click()

Извлекаем исходный код страницы и вспоминаем, как у нас извлекалась информация о пользователях в прошлый раз:

In [ ]:
html = br.page_source
soup = BeautifulSoup(html)
divs = soup.find_all("div", {"class" : "labeled name"})
L = [get_person(d) for d in divs] 

Осталась одна проблема – мы извлекли далеко не все результаты, а только те результаты, которые «видит» браузер, то есть то, что мы как пользователи видим до скроллинга. Библиотека `selenium` умеет скроллить страницы, точнее, активировать запуск кода на JavaScript, который отвечает за скроллинг. В общем виде строка с кодом для скроллинга выглядит так (`Y` – на сколько пикселей нужно проскроллить):

    br.execute_script("window.scrollTo(0, Y)") 
    
Если нужно проскроллить до конца страницы, то тогда вместо `Y` нужно вписать значение, которое извлекается из тела документа HTML:

     document.body.scrollHeight

Например, проскроллим текущую страницу до самого низа:

In [ ]:
br.execute_script("window.scrollTo(0, document.body.scrollHeight);")

Далее, чтобы открыть новые результаты, нам нужно найти кнопку *Показать ещё*. Если посмотрим внимательно на исходный код, найдём id этой кнопки:

In [ ]:
button_more = br.find_element(By.ID, "ui_search_load_more")

Кликаем:

In [ ]:
button_more.click()

Теперь нам нужно сделать следующее: выгружать информацию из исходного кода страницы по новым пользователям и скроллить страницу дальше. И повторять эти действия до тех пор, пока результаты не закончатся. Для начала напишем функцию `get_users_info()`, которая принимает на вход объект `br` (окно браузера), считывает исходный код и возвращает список с «чистыми» результатами – список пар *(имя пользователя, ссылка на профиль)*.

### Задача 1

Напишите функцию `get_users_info()` согласно описанию выше.

In [ ]:
### YOUR CODE HERE ###

Теперь напишем код (адаптированный ответ с [StackOverflow](https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python)) для цикла. Какой тип цикла нам нужен? Нам нужен цикл, который умеет повторять операции до тех пор, пока мы не дойдём до последнего результата, то есть до того момента, когда скроллить будет некуда. Воспользуемся конструкцией `while True`, бесконечным вариантом цикла `while`, который будет запускаться до тех пор, пока не дойдёт до кода с оператором `break` (выход из цикла) или не столкнётся с ошибкой. 

In [ ]:
# результатов много, фильтры очень общие
# оставновим цикл сами через Стоп (Kernel – Interrupt)

all_results = []
last_height = br.execute_script("return document.body.scrollHeight")

while True:
    res = get_users_info(br)
    all_results.extend(res) 
    
    br.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    new_height = br.execute_script("return document.body.scrollHeight")
    sleep(1)
    
    if new_height == last_height:
        break
        
    last_height = new_height

Пояснения к коду.

1. Сохраняем в переменную `last_height` величину, на которую мы можем проскроллить страницу за один раз в данный момент времени, чтобы дойти до конца страницы. 

2. На каждой итерации цикла `while` мы выгружаем информацию, добавляем её в список `all_results` и скроллим страницу до самого низа. После скроллинга проверяем, на сколько ещё можно проскроллить, сохраняем полученное значение в `new_height`. 

3. Если скроллить уже некуда, если мы находимся в самом низу «бесконечной» страницы с результатами поиска, то `new_height` совпадает с `last_height`. Значит, нам нужно остановить исполнение кода – выходим из цикла с помощью `break`. 

4. Если мы ещё не закончили скроллить, обновляем значение `last_height`, заменяя его на `new_height` (теперь уже в нём хранится величина, на которую мы можем проскроллить страницу за один раз в данный момент времени). Продолжаем выполнять выгрузку информации и скроллинг.

### Задача 2

Создайте новый список `results_unique`, который не содержит повторений (набор уникальных пар значений).

In [ ]:
### YOUR CODE HERE ###

### Задача 3

Создайте датафрейм на основе списка `results_unique`, присвойте столбцам подходящие названия и выгрузите таблицу в файл Excel.

In [ ]:
### YOUR CODE HERE ###

## Часть 2: выгрузка информации из ссылок в адресной строке

### Задача 1

Напишите функцию `get_link()`, которая принимает на вход строку с адресом, заходит на страницу [Google Maps](https://www.google.com/maps), вводит в поле для поиска этот адрес и сохраняет ссылку, которая скрывает в себе координаты, соответствующие этому адресу.

In [ ]:
### YOUR CODE HERE ###

### Задача 2

Примените эту функцию для всех адресов в списке `addresses` и создайте словарь с парами, где ключом является адрес, а значением – ссылка с координатами. Преобразуйте полученный словарь в датафрейм. 

In [ ]:
### YOUR CODE HERE ###

### Задача 3

Добавьте в полученный датафрейм столбцы с широтой и долготой, соответствующими каждому адресу.

In [ ]:
### YOUR CODE HERE ###